In [33]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

import pandas as pd
from IPython.display import display, Markdown, Latex

In [1]:
# Use a service account
cred = credentials.Certificate('./game-generation-6db9c-aef76f1917f8.json')
firebase_admin.initialize_app(cred)

db = firestore.client()

In [62]:
COLLECTION_NAME = 'participants-v2'
collection = db.collection(COLLECTION_NAME)

KEYS_TO_EXTRACT = ('participantID', 'timestamp', 'scene', 
    'game.setup', 'game.gameplay', 'game.scoring', 'game.difficulty', 'game.firstTimeScore', 
    'gameScore.score', 'gameScore.thoughts', 
    'debrief.strategy', 'debrief.difficulties', 'debrief.questions', 'debrief.external_aids')

SHORT_SCENE_NAMES = {
    'FloorPlan326_physics_semi_sparse_few_new_objects': 'few_objects',
    'FloorPlan326_physics_semi_sparse_new_objects': 'medium_objects',
    'FloorPlan326_physics_semi_sparse_many_new_objects': 'many_objects',
}

GAME_TEMPLATE = """(define (game {participantID}) (:domain {room}-objects-room-v1)  ; {index}
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))"""

def recursive_extract_value(d, key):
    if '.' in key:
        split_key = key.split('.')

    else:
        split_key = [key]

    value = d
    for key_part in split_key:
        if key_part == 'game':
            key_part = 'editedGame' if 'editedGame' in value else 'initialGame'
                
        value = value[key_part] if key_part in value else None
        if value == None:
            return value

    return value

def participant_dict_to_row(doc, keys=KEYS_TO_EXTRACT):
    d = doc.to_dict()
    return [doc.id] + [recursive_extract_value(d, key) for key in keys]


def print_participant(df, index, game_fields=('game_setup', 'game_gameplay', 'game_scoring')):
    p = df.loc[index]
    display(Markdown(f'## {p.participantID} ({p.id}) ({p.scene})'))
    for game_field in game_fields:
        display(Markdown(f'### **{game_field}:** {p[game_field]}'))

    room = p.scene.split('_')[0] if p.scene is not None else ''
    print(GAME_TEMPLATE.format(participantID=p.participantID, room=room, index=index))

In [80]:
participants_with_replays = collection.order_by('timestamp', direction=firestore.Query.DESCENDING).order_by('replays', direction=firestore.Query.DESCENDING).stream()
participant_rows = [participant_dict_to_row(doc) for doc in participants_with_replays]
participant_df = pd.DataFrame(participant_rows, columns=['id'] + [key.replace('.', '_') for key in KEYS_TO_EXTRACT])
participant_df.scene = [SHORT_SCENE_NAMES[s] if s in SHORT_SCENE_NAMES else None for s in participant_df.scene]
participant_df = participant_df[participant_df.scene.notna() & participant_df.game_setup.notna() & participant_df.game_gameplay.notna() & participant_df.game_scoring.notna()]
participant_df = participant_df.reset_index()

In [81]:
participant_df.head(10)

,index,id,participantID,timestamp,scene,game_setup,game_gameplay,game_scoring,game_difficulty,game_firstTimeScore,gameScore_score,gameScore_thoughts,debrief_strategy,debrief_difficulties,debrief_questions,debrief_external_aids
0,0,qSbsQv2jOptuDovotsIE,5e2df2855e01ef3e5d01ab58,2021-11-09 15:42:14.716000+00:00,medium_objects,No setup needed.,"To play my game, you will use the building blo...","To score my game, you will get 1 point for eac...",2,5,6,It was a lot more difficult to stack the block...,Since you were very much limited by the object...,The frame rate could get low at times.,It would have been nice to be able to simply ...,No.
1,1,SdpyTxFIhhYu7wNwmHFa,60e93f64ec69ecdac3107555,2021-11-09 15:38:30.625000+00:00,medium_objects,"The player should move basketball, beachball a...",The player stands next to the door facing the ...,If the ball was succesfully thrown into the bi...,2,7 points,4 points,I think it is interesting but requires practic...,I considered the games I like playing myself -...,I did not have any technical difficulties!,I found this study to be one of the most inter...,I used pen and paper to write down the items I...
2,2,PJ2WCWCLedT8sDLrvn49,613e4bf960ca68f8de00e5e7,2021-11-08 21:55:19.879000+00:00,medium_objects,,The pieces on the shelf between the two Window...,For each castle built in the correct order wil...,1,40,0,In real life this game would be very easy (tha...,I just looked at what was in the room and deci...,"Yes, the page broke and I had to redo it.",No,Print on a piece of paper the shape of the cas...
3,3,GLPtcvJUaHkUYK7iEPRq,613e4bf960ca68f8de00e5e7,2021-11-08 21:14:13.145000+00:00,medium_objects,,The pieces on the shelf between the two window...,For each castle built in the correct order of ...,0,40,0,In real life this game would be very easy (tha...,None,None,None,None
4,4,yWwOhJSjWiuzNkkHzhft,616e4f7a16145200573161a6,2021-11-08 21:10:48.702000+00:00,few_objects,closed blinds\nthe curved ramp with the square...,roll the dodge ball onto the bin across the ra...,if a ball enters the box its a point,3,maybe 1 point in 5 tries,0,its too complicated i couldnt play well,tried to move the objects to where I needed to...,the objects were hard to move around \nI could...,none,took screenshot
5,6,5DOVQT0dE9jCRwYHvIx6,5f5d6c3cbacc025bf0a03440,2021-11-08 16:54:14.336000+00:00,few_objects,First the bin should be turned upside down and...,The game that can be played is to use the dodg...,The scoring system consists in that each cubeb...,2,6,5,Nothing to add.,The strategy I used was to observe around the ...,The controls on the platform were a little clu...,No question or comment to add.,I did not use any external aid to carry out th...
6,8,oevKPEL8LXqT4gswKdMK,609c15fd6888b88a23312c42,2021-11-08 16:02:34.927000+00:00,medium_objects,,"Player has to throw dodgeballballs, basketball...",Every scored ball is 1 point. Every 3 balls sc...,1,3,2 points,My game is really hard because the physics in ...,None,None,None,None
7,9,Q9FPLgdunp3o75RFGEU7,616e5ae706e970fe0aff99b6,2021-11-08 15:58:09.085000+00:00,many_objects,you put the small ramp towards the center of t...,you roll the golf balls and the dodgeballs tow...,dodgeballs count as 3 points and golfballs cou...,3,6,0,,"I like simple games, especially when I play al...",It was lagging a lot.,No,I took notes to remember the names of objects
8,12,gZ8ce0O2anOHAMWWJJZw,613bb29f16252362f4dc11a3,2021-11-04 20:23:20.848000+00:00,medium_objects,You put the bin in the midle of the room and p...,You position the wooden ramp in different ways...,"Each ""wooden ramp"" position You make You have ...",4,1,I scored 1,It is a lot harder when I'm not used to the ga...,Use a game people are familiar with using obje...,I accidentally turned off the browser using a ...,"Not really, but the bin is very buggy if You p...",Not really. But I had like 3 tries in the quiz.
9,14,rTVvF1dBAKE8juHCF69f,5d29412ab711e9001ab74ece,2021-11-04 20:13:03.030000+00:00,many_objects,No preparation requ

In [92]:
print_participant(participant_df, 9)

## 5d29412ab711e9001ab74ece (rTVvF1dBAKE8juHCF69f) (many_objects)

### **game_setup:** No preparation required

### **game_gameplay:** Play a game of tower building using colored blocks. Your objective is to build highest possible tower that is stable, starting with any single block and ending with any pyramid block. No other block then first one is allowed to touch floor or walls or any non block objects in the room.

### **game_scoring:** Your score is number of blocks in tower including first and last (pyramid block)

(define (game 5d29412ab711e9001ab74ece) (:domain many-objects-room-v1)  ; 9
(:setup (and 

))
(:constraints (and 

))
(:scoring maximize

))
